In [1]:
#!pip install transformers accelerate trl


Defaulting to user installation because normal site-packages is not writeable
  Using cached rich-13.8.1-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.9 MB 6.1 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/9.9 MB 6.9 MB/s eta 0:00:02
   ----------- ---------------------------- 2.9/9.9 MB 4.5 MB/s eta 0:00:02
   ---------------- ----------------------- 4.2/9.9 MB 5.1 MB/s eta 0:00:02
   ------------------------- -------------- 6.3/9.9 MB 6.0 MB/s eta 0:00:01
   ---------------------------- ----------- 7.1/9.9 MB 6.3 MB/s eta 0:00:01
   ----------------------------------- ---- 8.7/9.9 MB 6.0 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 6.2 MB/s 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.9.0 requires torch==1.8.0, but you have torch 2.4.1 which is incompatible.


In [10]:
import ast
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import PPOTrainer, PPOConfig
import pandas as pd
import torch
# Load dataset
dataset = pd.read_csv("otblend_data.csv")

# Parse the 'Order_Candidates' column and select a random candidate
def parse_and_select(order_candidates):
    # Convert the string version of the list back to a Python list of tuples
    candidates = ast.literal_eval(order_candidates)
    # Randomly select one tuple from the list
    selected_candidate = random.choice(candidates)
    return selected_candidate[1]  # Return only the list part

# Function to convert a list to concatenated base-100 loss
def list_to_base100(concat_list):
    return ''.join([f'{x:02}' for x in concat_list])

# Compute rewards by processing 'Order_Candidates' column
def compute_rewards(order_candidates_column):
    rewards = []
    for candidates in order_candidates_column:
        selected_list = parse_and_select(candidates)
        # Convert the list into the concatenated base-100 format
        reward = list_to_base100(selected_list)
        rewards.append(int(reward))
    return rewards

# PPO configuration
ppo_config = PPOConfig(
    model_name='meta-llama/Llama-3.2-1B',
    learning_rate=5e-6,
    log_with="wandb"
)

# Load pre-trained LLaMA model and tokenizer
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

# Initialize PPOTrainer
ppo_trainer = PPOTrainer(
    config=ppo_config,
    model=model,
    tokenizer=tokenizer
)

# Training loop
for batch in dataset:
    # Tokenize inputs
    queries = tokenizer(batch['Word1'], return_tensors='pt')
    
    # Generate model responses
    responses = model.generate(**queries)
    
    # Compute rewards using the 'Order_Candidates' column
    rewards = compute_rewards(dataset['Order_Candidates'])
    
    # Apply PPO step
    ppo_trainer.step(queries.input_ids, responses, rewards)

# Save the fine-tuned model
model.save_pretrained("./fine-tuned-llama")


C:\Users\jonvw\AppData\Roaming\Python\Python39\site-packages\trl\trainer\ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-1B.
401 Client Error. (Request ID: Root=1-66f66726-174e26a92195324c2b881c24;162ee13e-a32c-405a-ac8b-a239c8ba6c04)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B is restricted. You must have access to it and be authenticated to access it. Please log in.